In [1]:
import getpass
from arcgis.gis import GIS

print('Modules Loaded')

Modules Loaded


In [2]:
# Login to ArcGIS Online (AGOL) by requesting the user to input a username and password
# documentation on getpass: https://docs.python.org/3/library/getpass.html

def login_AGOL():
    # prompt the user to enter their AGOL username
    username = input("Enter your ArcGIS Online username:")
    
    # prompt the user to enter their AGOL password (without echoing)
    password = getpass.getpass("Enter your ArcGIS Onine password:")
    
    # create a GIS object and login to AGOL
    gis = GIS("https://www.arcgis.com", username, password)
    
    # return the object
    return gis

# run module
gis = login_AGOL()

# confirm user login 
print("Logged in as:", gis.users.me.username)

Enter your ArcGIS Online username:stacey_vernon_build
Enter your ArcGIS Onine password:········
Logged in as: stacey_vernon_build
